### Installation

In [1]:
pip install -q tensorflow tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\crisp\\Documents\\anaconda3\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



#### Imports

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

### Checking datasets

In [ ]:
print(tfds.list_builders())

### Getting data Infomation

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
print(info)

### Data Preparation

In [ ]:
train = tfds.load(name='rock_paper_scissors', split="train")
test = tfds.load(name='rock_paper_scissors', split='test')

### Iterating over data
> To iterate over a tensorflow dataset we do it as follows

In [ ]:
for data in train:
  print(data['image'], data['label'])
  break

### Creating a Numpy data
> We are going to scale our data and convert it to a nummpy array

In [ ]:
train_images = np.array([data['image'].numpy()/255 for data in train])
train_labels =np.array([data['label'].numpy() for data in train])
test_image = np.array([data['image'].numpy()/255   for data in test])
test_labels = np.array([data['label'].numpy() for data in test])

In [ ]:
train_images[0]

### Class Names
0 - Rock

1 - Paper

2 - Scissors

In [ ]:
class_names = np.array(["rock", "paper", "scissor"])

### Creating a NN

In [ ]:
input_shape = train_images[0].shape
input_shape

In [ ]:

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'),
    keras.layers.MaxPool2D((3,3)) ,
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.summary()

### Combiling the Model

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=.0001),
    metrics=["accuracy"],
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

### Fitting the ModeL

In [ ]:
EPOCHS = 5
BATCH_SIZE = 4
VALIDATION_SET = (test_image, test_labels)
history = model.fit(train_images, train_labels, epochs=EPOCHS, validation_data=VALIDATION_SET, batch_size=BATCH_SIZE)

### Model Evaluation Conclusion
Our model is performing perfect. The loss on the train_set is almost 0 as well as the validation loss. The accuracy on the train set is `100%` compared to `83%` accuracy on the test set.

> The model is just overtraining but giving us good results on the validation set.

### Making Predictions


In [ ]:
predictions = model.predict(test_image[:10])
for i, j in zip(predictions, test_labels[:10]):
  print(class_names[np.argmax(i)],"-------->", class_names[j])

### Tunning Hyper Parameters -- Keras-Tunner
* [Docs](https://www.tensorflow.org/tutorials/keras/keras_tuner)


### Installation

In [ ]:
pip install -q -U keras-tuner

### Importing

In [ ]:
import kerastuner as kt

In [ ]:
def model_builder(hp):

  model = keras.Sequential()
  #   we want the model to find the best unit and the activation function for the first layer for us
  model.add(keras.layers.Conv2D(hp.Int('units',  min_value=32, max_value=512, step=32),(3, 3), 
                                input_shape=input_shape, activation=hp.Choice('activation-fn',values=['relu', 'sgd'])))
  
  model.add(keras.layers.MaxPool2D((3,3)))
  model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
  model.add(keras.layers.MaxPool2D((2,2)))
  model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
  model.add(keras.layers.MaxPool2D((2,2)))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, activation='relu'))
  model.add(keras.layers.Dense(32, activation='relu'))
  model.add(keras.layers.Dense(3, activation='softmax'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     )

In [ ]:
tuner.search(train_images, train_labels, validation_data=VALIDATION_SET, epochs=EPOCHS, batch_size=BATCH_SIZE)

> That's basically how the `kerastunner` works